In [34]:
import os
import re

import emoji
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# tf and keras
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses
from tensorflow.keras.layers import (
    Dense,
    Embedding,
    GlobalAveragePooling1D,
    Dropout,
    TextVectorization,
    Input,
    Conv1D,
    LSTM,
    MaxPooling1D,
    Bidirectional,
)
from tensorflow.keras.models import Model
# import tensorflow_datasets as tfds

pd.set_option('display.max_colwidth', 100) 

In [56]:
def read_files():
    X_train = pd.read_csv('./data/final/X_train.csv')
    y_train = pd.read_csv('./data/final/y_train.csv')
    X_val = pd.read_csv('./data/final/X_val.csv')
    y_val = pd.read_csv('./data/final/y_val.csv')
    
    train_not_na_indices = (X_train['selftext'].notna())
    val_not_na_indices = (X_val['selftext'].notna())
    
    X_train = X_train[train_not_na_indices]
    X_val = X_val[val_not_na_indices]

    y_train = y_train[train_not_na_indices]
    y_val = y_val[val_not_na_indices]

    return X_train, y_train, X_val, y_val

In [4]:
# # try and except the TF tokenizer
# try:
#     tokenizer = tfds.features.text.Tokenizer()
# except AttributeError:
#     tokenizer = tfds.deprecated.text.Tokenizer()

# # create an instance of the Counter class
# token_counts = Counter()

# for example in data_tf_train:
#     tokens = tokenizer.tokenize(example[0].numpy()[0])
#     token_counts.update(tokens)

# print('Size of training vocabulary:', len(token_counts))

# UTILS

In [42]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)

# X_train_v = X_train['selftext'].apply(vectorize_text)

# Model 1: Tokenizer and pad_sequences

In [27]:
X_train, y_train, X_val, y_val = read_files()

In [32]:
X_train = X_train[X_train['selftext'].notna()]

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Tokenizer setup
max_vocab_size = 10000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(X_train['selftext'])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(X_train['selftext'])
# Pad sequences to ensure uniform length
max_sequence_length = 250
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

print(padded_sequences)


[[8183 1084    6 ...    4 2698 1748]
 [   0  852  354 ... 9675 1039  102]
 [   0    0    0 ... 1255   13   12]
 ...
 [   0    0    0 ...  217 2062  578]
 [   0    0    0 ... 3057    3  925]
 [   0    0    0 ...    0    0    0]]


In [34]:
embedding_dim=16

In [35]:
model1 = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    layers.LSTM(128, return_sequences=True),
    Dropout(0.5),
    layers.LSTM(64),
    Dropout(0.5),
    Dense(1) # regression doesn't use an activation function
])


# Compile the model
model1.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
model1.summary()

# Train the model
model1.fit(vectorized_text, y_train, epochs=5, batch_size=2, validation_data=(X_val, y_val), verbose=1)

2024-07-24 17:19:59.112489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-24 17:19:59.115413: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-24 17:19:59.116770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 16)           160000    
                                                                 
 lstm (LSTM)                 (None, 250, 128)          74240     
                                                                 
 dropout (Dropout)           (None, 250, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 283,713
Trainable params: 283,713
Non-trai

2024-07-24 17:19:59.668487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-24 17:19:59.671003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-24 17:19:59.673251: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

639/639 [==============================] - ETA: 0s - loss: 550785.1250 - mae: 87.0742

2024-07-24 17:35:34.321342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-24 17:35:34.324987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-24 17:35:34.327636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

639/639 [==============================] - 1017s 2s/step - loss: 550785.1250 - mae: 87.0742 - val_loss: 2039985.7500 - val_mae: 106.4994
Epoch 2/10
639/639 [==============================] - 953s 1s/step - loss: 549058.6250 - mae: 94.9498 - val_loss: 2038535.5000 - val_mae: 113.4155
Epoch 3/10
639/639 [==============================] - 893s 1s/step - loss: 548087.3750 - mae: 101.3081 - val_loss: 2037464.1250 - val_mae: 119.9477
Epoch 4/10
639/639 [==============================] - 949s 1s/step - loss: 547450.2500 - mae: 107.6480 - val_loss: 2036765.2500 - val_mae: 125.1221
Epoch 5/10
639/639 [==============================] - 902s 1s/step - loss: 547190.8750 - mae: 112.0389 - val_loss: 2036262.6250 - val_mae: 129.5090
Epoch 6/10
639/639 [==============================] - 861s 1s/step - loss: 546764.3750 - mae: 116.0151 - val_loss: 2036002.8750 - val_mae: 132.0827
Epoch 7/10
639/639 [==============================] - 892s 1s/step - loss: 547236.9375 - mae: 112.0981 - val_loss: 2036952.2

# Model 2: TextVectorization Layer, Basic Embedding Model, Two Hidden Dense Layers (64, 32)

In [77]:
X_train, y_train, X_val, y_val = read_files()

In [78]:
def get_vectorization_layer(df, column, max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    vectorize_layer = layers.TextVectorization(
        max_tokens=max_tokens,
        output_mode='int',
        output_sequence_length=output_sequence_length)

    df[column] = df[column].astype(str)
    vectorize_layer.adapt(df[column].values)

    return vectorize_layer

def get_embedding_layer(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    return Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)

def build_model_2(output_sequence_length=250):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = embedding_layer(inputs)

    x = GlobalAveragePooling1D()(x)

    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [79]:
text_data = tf.constant(X_train['selftext'].values)
vectorize_layer = get_vectorization_layer(X_train, 'selftext')
vectorized_text = vectorize_layer(text_data)
embedding_layer = get_embedding_layer()
embedded_text = embedding_layer(vectorized_text)

In [92]:
model_2 = build_model_2()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_2 = model_2.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping])
pd.DataFrame(history_2.history)

Epoch 1/5
12770/12771 [============================>.] - ETA: 0s - loss: 842049.0000 - mean_absolute_error: 125.7566 - accuracy: 0.1303

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [89]:
def build_full_model_2(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = GlobalAveragePooling1D()(x)

    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [91]:
text_data = tf.constant(X_train['selftext'].values)
vectorize_layer = get_vectorization_layer(X_train, 'selftext')
vectorized_text = vectorize_layer(text_data)

model_2_full = build_full_model_2()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_2 = model_2_full.fit(vectorized_text, y_train, epochs=5, batch_size=2,verbose=1, callbacks=[early_stopping])
pd.DataFrame(history_2.history)

Epoch 1/5
12771/12771 [==============================] - 196s 15ms/step - loss: 841388.4375 - mean_absolute_error: 125.7218 - accuracy: 0.1302
Epoch 2/5
12771/12771 [==============================] - 201s 16ms/step - loss: 820631.0625 - mean_absolute_error: 120.4370 - accuracy: 0.1305
Epoch 3/5
12771/12771 [==============================] - 198s 15ms/step - loss: 791760.9375 - mean_absolute_error: 119.7966 - accuracy: 0.1305
Epoch 4/5
12771/12771 [==============================] - 199s 16ms/step - loss: 754356.0625 - mean_absolute_error: 116.1621 - accuracy: 0.1305
Epoch 5/5
12771/12771 [==============================] - 193s 15ms/step - loss: 705820.1875 - mean_absolute_error: 113.2395 - accuracy: 0.1305


,loss,mean_absolute_error,accuracy
0,841388.4375,125.721809,0.130222
1,820631.0625,120.436974,0.130496
2,791760.9375,119.796585,0.130496
3,754356.0625,116.162071,0.130496
4,705820.1875,113.239540,0.130496


# Model 3: TextVectorization Layer, Convolutional NN Embedding Model, Two Hidden Dense Layers (64, 32)

In [ ]:
X_train, y_train, X_val, y_val = read_files()

In [25]:
def build_model_3(output_sequence_length=250):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))

    x = embedding_layer(inputs)
    
    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [26]:
text_data = tf.constant(X_train['selftext'].values)
vectorize_layer = get_vectorization_layer(X_train, 'selftext')
vectorized_text = vectorize_layer(text_data)
embedding_layer = get_embedding_layer()
embedded_text = embedding_layer(vectorized_text)

In [27]:
model_3 = build_model_3()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3 = model_3.fit(vectorized_text, y_train, epochs=5, batch_size=2, validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])
pd.DataFrame(history_3.history)

Epoch 1/5
10216/10216 [==============================] - 336s 33ms/step - loss: 547076.2500 - mean_absolute_error: 126.3061 - accuracy: 0.1295 - val_loss: 2035162.8750 - val_mean_absolute_error: 145.3031 - val_accuracy: 0.1333
Epoch 2/5
10216/10216 [==============================] - 322s 31ms/step - loss: 546597.9375 - mean_absolute_error: 130.1261 - accuracy: 0.1295 - val_loss: 2035472.1250 - val_mean_absolute_error: 141.1581 - val_accuracy: 0.1333
Epoch 3/5
10216/10216 [==============================] - 367s 36ms/step - loss: 546377.3750 - mean_absolute_error: 129.5938 - accuracy: 0.1295 - val_loss: 2035215.1250 - val_mean_absolute_error: 143.3732 - val_accuracy: 0.1333
Epoch 4/5
10216/10216 [==============================] - 455s 45ms/step - loss: 546211.4375 - mean_absolute_error: 130.3387 - accuracy: 0.1295 - val_loss: 2036054.5000 - val_mean_absolute_error: 142.9070 - val_accuracy: 0.1333
Epoch 5/5
10216/10216 [==============================] - 401s 39ms/step - loss: 545547.8125 

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,547076.2500,126.306091,0.129523,2035162.875,145.303070,0.133294
1,546597.9375,130.126083,0.129454,2035472.125,141.158066,0.133294
2,546377.3750,129.593842,0.129453,2035215.125,143.373245,0.133294
3,546211.4375,130.338745,0.129454,2036054.500,142.906967,0.133294
4,545547.8125,130.453171,0.129454,2037564.625,121.825531,0.133294


In [29]:
pd.DataFrame(history3.history)

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,547076.2500,126.306091,0.129523,2035162.875,145.303070,0.133294
1,546597.9375,130.126083,0.129454,2035472.125,141.158066,0.133294
2,546377.3750,129.593842,0.129453,2035215.125,143.373245,0.133294
3,546211.4375,130.338745,0.129454,2036054.500,142.906967,0.133294
4,545547.8125,130.453171,0.129454,2037564.625,121.825531,0.133294


# Model 4: TextVectorization Layer, LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [ ]:
X_train, y_train, X_val, y_val = read_files()

In [ ]:
def build_model_4(output_sequence_length=250):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))

    x = embedding_layer(inputs)
    
    x = layers.LSTM(32, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [ ]:
text_data = tf.constant(X_train['selftext'].values)
vectorize_layer = get_vectorization_layer(X_train, 'selftext')
vectorized_text = vectorize_layer(text_data)
embedding_layer = get_embedding_layer()
embedded_text = embedding_layer(vectorized_text)

In [ ]:
model_4 = build_model_4()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_4 = model_4.fit(vectorized_text, y_train, epochs=5, batch_size=2, validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])
pd.DataFrame(history_4.history)

# Model 5: TextVectorization Layer, Bi-Directional LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [57]:
X_train, y_train, X_val, y_val = read_files()

In [58]:
def build_model_5(output_sequence_length=250):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))

    x = embedding_layer(inputs)
    
    x = Bidirectional(LSTM(32, activation='relu'))(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [59]:
text_data = tf.constant(X_train['selftext'].values)
vectorize_layer = get_vectorization_layer(X_train, 'selftext')
vectorized_text = vectorize_layer(text_data)
embedding_layer = get_embedding_layer()
embedded_text = embedding_layer(vectorized_text)

In [60]:
model_5 = build_model_5()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_5 = model_5.fit(vectorized_text, y_train, epochs=5, batch_size=2, validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])
pd.DataFrame(history_5.history)

Epoch 1/5
   86/12771 [..............................] - ETA: 1:30:12 - loss: nan - mean_absolute_error: nan - accuracy: 0.1628                                     

KeyboardInterrupt: 

# Using TextVectorization Layer 2 

In [27]:
vocab_size = len(vectorize_layer.get_vocabulary())

# Input shape:  (batch_size, input_length)
# Output shape: (batch_size, input_length, output_dim)
embedding_layer = tf.keras.layers.Embedding(
    input_dim = vocab_size,  # size of feature vocabulary
    output_dim = 2,   # embedding dimension
    input_length = max_sequence_length  # number of inputs
    )

first_review_embed_rep = embedding_layer(X_train_v[0])

In [29]:
def build_model(vectorize_layer):
    vocab_size = len(vectorize_layer.get_vocabulary())

    tf.keras.backend.clear_session()
    tf.random.set_seed(0)

    model = tf.keras.Sequential()
    model.add(vectorize_layer)
    model.add(tf.keras.layers.Embedding(
        input_dim = vocab_size,  # size of feature vocabulary
        output_dim = 2,  # embedding dimension
        input_length = max_sequence_length  # number of inputs
        ))

    # Average over the sequence dimension, so each review is represented by
    # 1 vector of size embedding_dimension
    model.add(tf.keras.layers.GlobalAveragePooling1D())

    # Alternatively, we could concatenate the embedding representations of
    # all tokens in the movie review
    #model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(
      units=8,
      activation='relu'))

    model.add(tf.keras.layers.Dense(
      units=1,
      activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    return model

In [30]:
def get_vectorization_layer(df, column, max_features=10000, sequence_length=250, embedding_dim=16):
    vectorize_layer = layers.TextVectorization(
        max_tokens=max_features,
        output_mode='int',
        output_sequence_length=sequence_length)

    df[column] = df[column].astype(str)
    vectorize_layer.adapt(df[column])

    return vectorize_layer

vectorize_layer = get_vectorization_layer(X_train, 'selftext')

model = build_model(vectorize_layer)

# Display the model layers.
display(model.layers)
display(model.summary())

# Retrieve the embeddings layer, which itself is wrapped in a list.
embeddings = model.layers[1].get_weights()[0]
print('|'*100)
display("Embeddings layer - shape: ", embeddings.shape)
print('|'*100)
display("Embeddings layer - parameter matrix (before training): ", embeddings)

ValueError: Exception encountered when calling layer "text_vectorization_1" (type TextVectorization).

When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, None) with rank=2

Call arguments received by layer "text_vectorization_1" (type TextVectorization):
  • inputs=tf.Tensor(shape=(None, None), dtype=string)

# End of file

Sources:
* https://stackoverflow.com/questions/73878049/how-do-you-convert-the-pandas-dataframe-to-tensorflow-python-data-ops-dataset-op